# Artificial Neural Network (ANN)

**Dataset**:
- Bank that collected customers info over the last 6 months and whether they left (`Exited=1`) or not (`Exited=0`)
- The Bank wants to understand the reasons for customers leaving and predict customers in high risk of leaving (to do preventive actions)

### Importing the libraries

In [131]:
import numpy as np
import pandas as pd
import tensorflow as tf

tf.__version__

'2.15.0'

## Part 1 - Data Preprocessing

### Importing the dataset

In [132]:
dataset = pd.read_csv('./filez/Churn_Modelling.csv')
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [133]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [134]:
# exclude rownum, customer id and name
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

print(f'X:\n{X[:3]}')
print(f'y:\n{y[:3]}')

X:
[[619 'France' 'Female' 42 2 0.0 1 1 1 101348.88]
 [608 'Spain' 'Female' 41 1 83807.86 1 0 1 112542.58]
 [502 'France' 'Female' 42 8 159660.8 3 1 0 113931.57]]
y:
[1 0 1]


### Encoding categorical data

In [135]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

# encode genre
X[:, 2] = le.fit_transform(X[:, 2])
print(X[:3])

[[619 'France' 0 42 2 0.0 1 1 1 101348.88]
 [608 'Spain' 0 41 1 83807.86 1 0 1 112542.58]
 [502 'France' 0 42 8 159660.8 3 1 0 113931.57]]


In [136]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

ct = ColumnTransformer(
    transformers=[("encoder", OneHotEncoder(), [1])], remainder="passthrough"
)
X = np.array(ct.fit_transform(X))

print(X[:3])

[[1.0 0.0 0.0 619 0 42 2 0.0 1 1 1 101348.88]
 [0.0 0.0 1.0 608 0 41 1 83807.86 1 0 1 112542.58]
 [1.0 0.0 0.0 502 0 42 8 159660.8 3 1 0 113931.57]]


### Splitting the dataset into the Train/Test set

In [137]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

### Feature Scaling
Mandatory in deep learning!

In [138]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Part 2 - Building the ANN

### Initializing the ANN

In [139]:
ann = tf.keras.models.Sequential()

### Adding the input layer and the 1st hidden layer
How many neurons? no rule of thumb. Start with one and then iterate.

In [140]:
# relu -> rectified activation function
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))


### Adding the 2nd layer hidden layer

In [141]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Adding the output layer

In [142]:
# - output is one binary value, so 1 output neuron.
# - for categorical output (eg: 'A', 'B', 'C'), we'd need
#   3 neurons and activation = `softmax` instead of `sigmoid`
ann.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

## Part 3 - Training the ANN

### Compiling the ANN

In [143]:
# optimizer: `adam` is based on Stochastic Gradient Descent
# loss: `binary_crossentropy` for binary classification
#       `categorical_crossentropy`for categorical classification
ann.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

### Training the ANN on the Training set

In [144]:
# `batch_size` is commonly set to 32
ann.fit(x=X_train, y=y_train, batch_size=32, epochs=100)

Epoch 1/100
  1/250 [..............................] - ETA: 51s - loss: 0.5765 - accuracy: 0.8125

250/250 [==============================] - 0s 575us/step - loss: 0.4989 - accuracy: 0.7960
Epoch 2/100
250/250 [==============================] - 0s 413us/step - loss: 0.4517 - accuracy: 0.7960
Epoch 3/100
250/250 [==============================] - 0s 414us/step - loss: 0.4340 - accuracy: 0.7960
Epoch 4/100
250/250 [==============================] - 0s 463us/step - loss: 0.4251 - accuracy: 0.7971
Epoch 5/100
250/250 [==============================] - 0s 446us/step - loss: 0.4193 - accuracy: 0.8144
Epoch 6/100
250/250 [==============================] - 0s 436us/step - loss: 0.4149 - accuracy: 0.8201
Epoch 7/100
250/250 [==============================] - 0s 420us/step - loss: 0.4109 - accuracy: 0.8219
Epoch 8/100
250/250 [==============================] - 0s 406us/step - loss: 0.4068 - accuracy: 0.8244
Epoch 9/100
250/250 [==============================] - 0s 420us/step - loss: 0.4023 - accuracy: 0.8305
Epoch 10/100
250/250 [==============================] - 0s 431us/step - loss: 0.3976 

## Part 4 - Making the predictions and evaluating the model

### Predicting the result of a single observation

In [145]:
y_pred_raw = [600, "France", 1, 40, 3, 60000.0, 2, 1, 1, 50000.0]

# 1) Transformation
# @dev: CAREFUL!, this doesn't work because there's only 1 category value
# y_pred_transformed = np.array(ct.fit_transform([y_pred_raw]))
# so doing it manually:
y_pred_transformed = [1, 0, 0, 600, 1, 40, 3, 60000.0, 2, 1, 1, 50000.0]

# 2) Scaling
y_pred_scaled = sc.transform([y_pred_transformed])

# 3) Predicting the probability that this customers leaves the bank
y_pred_customer = ann.predict([[y_pred_scaled]])

print(f'probability: {y_pred_customer[0][0]:.2%}')
print(f'will leave?: {y_pred_customer[0][0] > 0.5}')


1/1 [==============================] - 0s 30ms/step
probability: 3.96%
will leave?: False


### Predicting the Test set results

In [146]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test), 1)), 1))

63/63 [==============================] - 0s 382us/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


### Making the Confusion Matrix

In [147]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1522   73]
 [ 207  198]]


0.86

    Looks like there's room for improvement..